<a href="https://colab.research.google.com/github/RozitaAbdoli/credit_default_mining/blob/main/Model_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Comparing the performance of multiple ML algorithms
In this notebook, the performance of several ML algorithms including ensemble learners will be compared. The ensemble learners are Random Forest, Adaptive Boost, and Gradient Boost Algorithms that use Decision Trees as weak learners. They:  

* Are non-parametric and don't require data to be normally distributed.
* Can handle both numeric and categorical data types, and categorical predictors don't have to be one-hot encoded.
* The correlation between predictors does not affect the performance of the model.

In [46]:
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn import preprocessing
from imblearn.combine import SMOTETomek 

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
# from sklearn.neural_network import MLPClassifier
# from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score, f1_score

In [47]:
#Import Drive API and authenticate
from google.colab import drive
#Mount Drive to the Colab VM
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
#Load the dataset into pandas DataFrame
df = pd.read_csv("/content/drive/MyDrive/Capstone_project/v2_credit_default.csv")

In [49]:
# #  Selected features Based on Pearson and Spearman's rank correlations with the dependent variable, and XGBoost feature importance rankings:
# df = df[['AGE', 'LIMIT_BAL', 'Pay_Apr', 'Repay_Sept', 'Pay_Sept','Default' ]]

In [50]:
df.shape

(29965, 24)

In [51]:
#Seperate the independent and dependent variables.
df_independent = df.drop(['Default'], axis=1)
df_default = df['Default']

In [52]:
# split the data into 75% training+validation and 25% test
X_train, X_test, y_train, y_test = train_test_split(df_independent, df_default, test_size=0.25, random_state=1) 

In [53]:
# Scale X_train and X_test.
X_train_scaled = preprocessing.MinMaxScaler().fit_transform(X_train)
X_test_scaled = preprocessing.MinMaxScaler().fit_transform(X_test)

In [54]:
# Balance the training data using SMOTE Tomek
X_smt, y_smt = SMOTETomek(random_state=1).fit_sample(X_train_scaled, y_train.squeeze())
# X_smt, y_smt = SMOTETomek(random_state=1).fit_sample(X_train, y_train.squeeze())    #use this line, if skipping the scaling step

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [55]:
#List the classifiers to be compared
names = ["Logistic_Regression", "Decision_Tree","Nearest_Neighbours", "QDA", "Naive_Bayes", "Random_Forest", "AdaBoost", "GradientBoost"]  

clfs = [
    LogisticRegression(random_state=1, C= 50, penalty= 'l1', solver= 'liblinear'),
    DecisionTreeClassifier(max_depth=5),              #set max_depth 
    KNeighborsClassifier(),                           #n_neighbors=5 
    QuadraticDiscriminantAnalysis(),
    GaussianNB(),
    RandomForestClassifier(max_depth=5, n_estimators=100),
    AdaBoostClassifier(n_estimators=100), 
    GradientBoostingClassifier(n_estimators=100, learning_rate=1.0),           
    ]

In [56]:
accuracies = []
precisions = []
recalls = []

for name, clf in zip(names, clfs):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    precision = precision_score(y_test, y_pred)
    precisions.append(precision)
    recall = recall_score(y_test, y_pred)
    recalls.append(recall)

In [57]:
#Precision= TP/(TP+FP), Recall= TP/(TP+FN)
scores_df = pd.DataFrame()
scores_df['Algorithm'] = names
scores_df['Accuracy'] = accuracies
scores_df['Precision'] = precisions
scores_df['Recall'] = recalls
scores_df

,Algorithm,Accuracy,Precision,Recall
0,Logistic_Regression,0.813267,0.709845,0.250305
1,Decision_Tree,0.818473,0.659864,0.354446
2,Nearest_Neighbours,0.757742,0.385733,0.177832
3,QDA,0.618793,0.334424,0.746650
4,Naive_Bayes,0.391484,0.247533,0.870889
5,Random_Forest,0.812600,0.670000,0.285627
6,AdaBoost,0.815136,0.653892,0.332521
7,GradientBoost,0.804058,0.582857,0.372716


Some notes:

* Scaling inputs between 0 and 1 did not affect the performance of the models much.
* Can see the trade-off between precision and recall in this algorithms: when precision is high recall suffers, and vice versa.